# Multi GPU T4 Syntetic Data Process

In [ ]:
%%capture
!pip install torch
!pip install diffusers

In [ ]:
import random
import os
import threading
import torch
import boto3
from diffusers import DiffusionPipeline
from io import BytesIO
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("aws_access_key_id")
secret_value_1 = user_secrets.get_secret("aws_secret_access_key")

# Configurar cliente de S3
s3 = boto3.client(
    's3',
    aws_access_key_id=secret_value_0,
    aws_secret_access_key=secret_value_1
)
BUCKET_NAME = 'faces-bucket-emotion'  # Cambia esto al nombre de tu bucket de S3
DIRECTORIO_BASE = 'faces'
# Crear las carpetas base en S3 para cada emoción
emociones = ['happy', 'surprised', 'angry', 'sad']

# Cargar los pipelines
#--"RunDiffusion/RunDiffusion-XL"
#--"SG161222/RealVisXL_V4.0" (best)
pipeline0 = DiffusionPipeline.from_pretrained("SG161222/RealVisXL_V4.0", torch_dtype=torch.float16).to('cuda:0')
pipeline1 = DiffusionPipeline.from_pretrained("SG161222/RealVisXL_V4.0", torch_dtype=torch.float16).to('cuda:1')

In [ ]:
# Definir la función para ejecutar el pipeline y guardar la imagen en S3
def exe_pipe(pipeline, prompt, emotion, index, negative_prompt):
    print(f'Ejecutando prompt: {prompt}')
    img = pipeline(prompt, negative_prompt=negative_prompt).images[0]
    # Guardar la imagen en S3
    img_bytes = BytesIO()
    img.save(img_bytes, format='PNG')
    img_bytes.seek(0)
    s3_path = f'{DIRECTORIO_BASE}/{emotion}/{emotion}_{index}.png'
    s3.upload_fileobj(img_bytes, BUCKET_NAME, s3_path)
    print(f'Imagen guardada en S3: {s3_path}')

# Definir las etnicidades y géneros
ethnicities = ['a latino', 'a white', 'a black', 'a middle eastern', 'an indian', 'an asian']
genders = ['male', 'female']

# Definir los prompts de emoción
emotion_prompts = {
    'happy': 'smiling',
    'surprised': 'surprised, opened mouth, raised eyebrows',
    'angry': 'angry',
    'sad': 'frowning, sad face expression, crying'
}

# Definir el negative prompt para evitar ciertos elementos no deseados
negative_prompts = [
    "blurry image", 
    "low quality", 
    "bad lighting", 
    "distorted face", 
    "deformed hands", 
    "extra limbs", 
    "unnatural skin tones"
]

def generar_caras(pipeline, emociones):
    for emotion in emociones:
        for i in range(50):
            ethnicity = random.choice(ethnicities)
            gender = random.choice(genders)
            emotion_prompt = emotion_prompts[emotion]

            prompt = f"Medium-shot portrait of {ethnicity} {gender}, {emotion_prompt}, front view, looking at the camera, color photography, photorealistic, hyperrealistic, realistic, incredibly detailed, crisp focus, digital art, depth of field, 50mm, 8k"

            # Ejecutar el pipeline y guardar la imagen, ahora con negative_prompt
            exe_pipe(pipeline, prompt, emotion, i, negative_prompt=", ".join(negative_prompts))



In [ ]:
# Crear y ejecutar los hilos
hilo0 = threading.Thread(target=generar_caras, args=(pipeline0, ['happy', 'surprised']))
hilo1 = threading.Thread(target=generar_caras, args=(pipeline1, ['angry', 'sad']))

hilo0.start()
hilo1.start()